# Script to Fetch Examples via Openrouter 

# (0) General Script Setup

In [ ]:
# (0) Import required libraries
import csv
import json
import keyring
import os
from datetime import datetime
from openrouter import OpenRouter

# (1) Define constants
SERVICE_NAME = "openrouter"        # keyring service identifier
KEYRING_USERNAME = "api_key"       # keyring username for the API key
OUTPUT_CSV = "results.csv"         # path to the CSV file for API call results

# (2) API Key Management
set_key = lambda key: keyring.set_password(SERVICE_NAME, KEYRING_USERNAME, key)
get_key = lambda: keyring.get_password(SERVICE_NAME, KEYRING_USERNAME)


## (1) Setup to make OpenRouter API Calls

In [ ]:
# (1) Client factory
def get_client() -> OpenRouter:
    """Aim: Create an OpenRouter client using the stored API key from keyring.
    Input: None.
    Output: OpenRouter — an initialised client instance.
    """
    api_key = get_key()
    if api_key is None:
        raise ValueError("No OpenRouter API key found in keyring. Please set it using set_key().")
    return OpenRouter(api_key=api_key)


# (2) Low-level call helper
def call_openrouter(
    messages: list[dict],
    model: str = "openai/gpt-4.1",
    temperature: float = 1.0,
    max_tokens: int | None = None,
    response_format: dict | None = None,
) -> dict:
    """Aim: Send a chat-completion request to OpenRouter.
    Input: messages (list[dict]), model (str), temperature (float),
           max_tokens (int|None), response_format (dict|None).
    Output: Response object from the API.
    """
    kwargs = dict(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    if max_tokens is not None:
        kwargs["max_tokens"] = max_tokens
    if response_format is not None:
        kwargs["response_format"] = response_format

    with get_client() as client:
        response = client.chat.send(**kwargs)
    return response


# (3) Extract assistant text from a response
def extract_content(response) -> str:
    """Aim: Extract the assistant's text from a completions response.
    Input: response — API response object.
    Output: str — the assistant's message content.
    """
    return response.choices[0].message.content


# (4) Quick sanity check – uncomment to test:
# resp = call_openrouter([{"role": "user", "content": "Say hello!"}])
# print(extract_content(resp))


# (2) Setup to build the prompt

In [ ]:
# (1) Define the dynamic parts used in the persuasive prompt stem
issue_stance = "Universal healthcare should be adopted in the United States."
epistemic_category_description = (
    "You believe this stance is justified based on empirical evidence and moral reasoning."
)
tactic_description = (
    "providing clear arguments, citing evidence, and appealing to both logic and emotion"
)

# (2) The prompt stem with placeholders for semantic elements
# (a) Define a prompt stem that can be used to generate examples for the persuasion taxonomy
prompt_stem = (
    "Write a 50-word persuasive example supporting the following stance: "
    "‘{issue_stance}’. {epistemic_category_description}. "
    "Use the following approach: {tactic_description}. "
    "Produce only the example text."
)

# (b) Define a prompt stem that can be used to test prompts for RCT users.  
# prompt_stem = (
#     "You’re conversing with a user and your primary goal is, over the course of the conversation, to\n\n"
#     "persuade the user to support the following issue stance: ‘{issue_stance}’. {epistemic_category_description}. "
#     "In particular, you should convince the user by {tactic_description}. If the user already agrees with this\n\n"
#     "issue stance, your goal is to try to get them to agree with it *EVEN MORE*. Do NOT tell the user that\n\n"
#     "your goal is to persuade them. Start the conversation by saying Hello to the user, and then start persuading.\n\n"
#     "Don’t generate the full conversation; just start it."
# )

# (3) Compose the full prompt for the LLM using the elements above.
def build_prompt():
    """
    Aim: Compose the full prompt for the LLM using the elements above.
    Output: str — the ready-to-send prompt for OpenRouter.
    """
    return prompt_stem.format(
        issue_stance=issue_stance,
        epistemic_category_description=epistemic_category_description,
        tactic_description=tactic_description,
    )


# (4) Wrap the prompt into the chat-messages format expected by the API.
def build_messages(prompt: str) -> list[dict]:
    """
    Aim: Wrap the composed prompt into the chat-messages list expected by call_openrouter.
    Input: prompt (str) — the full prompt text produced by build_prompt().
    Output: list[dict] — a messages list with a single user message.
    """
    return [{"role": "user", "content": prompt}]

# (3) Make API calls & export results

In [ ]:
# (1) Make a single API call and append the result to a CSV
def make_api_call(model_name: str, csv_path: str = OUTPUT_CSV) -> None:
    """
    Build the prompt using build_prompt, call the LLM API, and append the result to a CSV.
    CSV has columns: issue_stance, epistemic_category_description, tactic_description, model_name, llm_response
    """
    prompt = build_prompt()
    msgs = build_messages(prompt)
    resp = call_openrouter(msgs, model=model_name)
    llm_response = extract_content(resp)

    row = {
        "issue_stance": issue_stance,
        "epistemic_category_description": epistemic_category_description,
        "tactic_description": tactic_description,
        "model_name": model_name,
        "llm_response": llm_response
    }

    file_exists = os.path.isfile(csv_path)
    with open(csv_path, mode="a", newline='', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=row.keys())
        if not file_exists:
            writer.writeheader()
        writer.writerow(row)
    print(f"LLM response saved to {csv_path}")


# (2) Run a single call using the globals already defined in section (2)
# make_api_call("openai/gpt-4.1")

